## Cube splitting in tetrahedrons

In [6]:
from Bio.PDB import PDBIO, Structure, Model, Chain, Atom, Residue
from Bio.PDB.PDBIO import Select

import py3Dmol
import fileinput
import os

file_name_5 = "cube_5_tetrahedra"
file_name_6 = "cube_6_tetrahedra"

dispositions = [file_name_5, file_name_6]

### Setup and functions
Define the atoms coordinates (cube), the connectivity of the cube and the coordinates of the tetrahedrons for the two ways of split

In [7]:
############### MY ATOM ###############

type = "elia"

# Define cube vertices (pseudoatoms)
atoms = {
    "PS1": (0.0, 0.0, 0.0),
    "PS2": (10.0, 0.0, 0.0),
    "PS3": (0.0, 0.0, 10.0),
    "PS4": (10.0, 0.0, 10.0),
    "PS5": (0.0, 10.0, 0.0),
    "PS6": (10.0, 10.0, 0.0),
    "PS7": (0.0, 10.0, 10.0),
    "PS8": (10.0, 10.0, 10.0),
}

# Define connectivity (edges of the cube)
cube_connectivity = [
    (1, 2), (1, 3), (1, 5),
    (2, 4), (2, 6),
    (3, 4), (3, 7),
    (4, 8),
    (5, 6), (5, 7),
    (6, 8),
    (7, 8),
]

cube_5_tetrahedra = [
    (1,2,3,5),
    (2,5,6,8),
    (2,3,4,8),
    (3,5,7,8),
    (2,5,3,8),
]

cube_6_tetrahedra = [
    (1,3,4,5),
    (1,2,4,5),
    (2,5,6,4),
    (5,6,4,8),
    (7,8,4,5),
    (7,3,4,5),
]

cube_6_connectivity = [
    (5,3),(1,4),(5,4),
    (5,2),
    (4,6),
    (5,8),
    (4,7),(8,5),
    (7,4),
]


In [8]:
############# JAPANISE ATOM #############

type = "jpn"

# Define connectivity (edges of the cube)
cube_connectivity = [
    (1, 2), (1, 3), (1, 5),
    (2, 4), (2, 6),
    (3, 4), (3, 7),
    (4, 8),
    (5, 6), (5, 7),
    (6, 8),
    (7, 8),
]

atoms = {
    "PS1": (10.0, 10.0, 10.0),
    "PS2": (0.0, 10.0, 10.0),
    "PS3": (10.0, 0.0, 10.0),
    "PS4": (0.0, 0.0, 10.0),
    "PS5": (10.0, 10.0, 0.0),
    "PS6": (0.0, 10.0, 0.0),
    "PS7": (10.0, 0.0, 0.0),
    "PS8": (0.0, 0.0, 0.0),
}

cube_5_tetrahedra = [
    (4, 6, 7, 8),
    (1, 5, 6, 7),
    (1, 3, 4, 7),
    (1, 2, 4, 6),
    (1, 4, 6, 7),
]

cube_6_tetrahedra = [
    (3,4,6,8),
    (3,6,7,8),
    (3,5,6,7),
    (1,3,5,6),
    (1,2,3,6),
    (2,3,4,6),
]

In [9]:
def add_connectivity(name):
    """function from list of vertices of tetrahedra to list of couples of vertices. This way
        it's easier to plot a cube with its tetrahedra
    Args:
        name (list): list of tetrahedron vetrices

    Returns:
        list: list of couples of vertices, each one define a vertex that compose a tetrahedron
    """
    tetrahedra_connectivity = []
    for tetrahedron in name:
        for i in range(4):
            for j in range(i+1, 4):
                tetrahedra_connectivity.append((tetrahedron[i], tetrahedron[j]))

    return tetrahedra_connectivity

# Create structure
def get_structure(name):
    """return the structure of a cube, putting atoms in the vertices

    Args:
        name (string): name of the structure

    Returns:
        Structure: Structure of the cube with atoms at its vertices
    """
    structure = Structure.Structure(name)
    model = Model.Model(0)
    chain = Chain.Chain("A")
    structure.add(model)
    model.add(chain)

    # Add atoms
    for i, (name, coords) in enumerate(atoms.items(), start=1):
        res = Residue.Residue((" ", i, " "), "PSE", " ")
        atom = Atom.Atom(name, coords, 1.0, 1.0, " ", name, i, "C")
        res.add(atom)
        chain.add(res)

    return structure


def postprocess(stream, connectivity):
    """process a connectivity list and add it to the pdb file

    Args:
        stream (Stream): os stream to file writing 
        connectivity (List): list describing the connectivity of each edge in a couple of vertices way 
    """
    added = []
    for a1, a2 in connectivity:
        added.append((a1, a2))

    set_added = set(added)
    new = list(set_added)
    new.sort()

    for (a1, a2) in new:
        if(a2, a1) in new:
            set_added.remove((a1, a2))

    stream.write("\n")
    for a1, a2 in new:
        stream.write(f"CONECT{a1:5}{a2:5}\n")

def sort_connectivity(connectivity):
    """Sort every couple of the connectivity so that VMD can see it in PDB file. CONECT n m must have n < m

    Args:
        connectivity (List): list of connected verteces creating edges

    Returns:
        list: sorted list of sorted couples of vertices
    """
    sorted_list = []
    for a1, a2 in connectivity:
        if(a1 > a2):
            a1, a2 = a2, a1
        sorted_list.append((a1, a2))

    return sorted(set(list(sorted_list)))

def clean(cube_connectivity, tetrahedra_connectivity_sorted):
    """Remove the elements already in file because shared

    Args:
        cube_connectivity (list): list of couples of vertices describing the cube connectivity
        tetrahedra_connectivity_sorted (list): list of couples of vertices describing the tetrahedra connectivity at its best

    Returns:
        list: list of couples of vertices describing the tetrahedra connectivity without repetitions
    """
    for edge in cube_connectivity:
        if edge in tetrahedra_connectivity_sorted:
            tetrahedra_connectivity_sorted.remove(edge)
    
    return tetrahedra_connectivity_sorted


### Cube creation
Create the cube with its splits

In [10]:
CUBE_ONLY = False                                                                                       # Only save the cube

if CUBE_ONLY:
    dispositions = ["cube"]                             
else:
    dispositions = [file_name_5, file_name_6]

for file_name in dispositions:
    # Create the 8 atoms for the cube
    io = PDBIO()                                                                                        # Create the molecule OBJ
    io.set_structure(get_structure(file_name))
    io.save(file_name + ".pdb")                                                                         # Save it as PDB

    with open(file_name + ".pdb", "r") as file:                                                         # Read all lines
        lines = file.readlines()        
                                         
    with open(file_name + ".pdb", "w") as file:                                                         # Write back only lines that do NOT start with "TER" or "END"
        for line in lines:
            if not (line.startswith("TER") or line.startswith("END")):
                file.write(line)

    with open(file_name + ".pdb", "a") as stream:
        postprocess(stream, cube_connectivity)                                                          # Connect the cube vertices
        if not CUBE_ONLY:
            tetrahedra_connectivity = add_connectivity(eval(file_name))                                 # Create the coupled way connectivity
            tetrahedra_connectivity_sorted = sort_connectivity(tetrahedra_connectivity)                 # Sort it
            tetrahedra_connectivity_clean = clean (cube_connectivity, tetrahedra_connectivity_sorted)   # Clean it from the doubled values
            postprocess(stream, tetrahedra_connectivity_sorted)                                         # Add it to the PDB file
        stream.write("END\n")                                                                           # End the molecule

    print("PDB file with CONECT records saved as", file_name + ".pdb")



PDB file with CONECT records saved as cube_5_tetrahedra.pdb
PDB file with CONECT records saved as cube_6_tetrahedra.pdb


### Simple plot
Follow a simple way to easily see the results, plotting also the axes

In [11]:
for file_name in dispositions:
    # Read a pdb file
    with open((file_name + ".pdb"), "r") as f:
        pdb_data = f.read()

    # Visualize in Jupyter Notebook
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(pdb_data, "pdb")

    # Style stick, add color
    viewer.setStyle({"stick": {'colorscheme': "cyanCarbon"}})

    # Add X, Y, Z axes using cylinders
    axis_length = 20
    axis_radius = 0.2

    # X-axis (Red)
    viewer.addCylinder({
        'start': {'x': 0, 'y': 0, 'z': 0}, 
        'end': {'x': axis_length, 'y': 0, 'z': 0}, 
        'radius': axis_radius, 
        'color': 'red'
    })

    # Y-axis (Green)
    viewer.addCylinder({
        'start': {'x': 0, 'y': 0, 'z': 0}, 
        'end': {'x': 0, 'y': axis_length, 'z': 0}, 
        'radius': axis_radius, 
        'color': 'green'
    })

    # Z-axis (Blue)
    viewer.addCylinder({
        'start': {'x': 0, 'y': 0, 'z': 0}, 
        'end': {'x': 0, 'y': 0, 'z': axis_length}, 
        'radius': axis_radius, 
        'color': 'blue'
    })

    viewer.zoomTo()
    # viewer.show()                                             # Uncomment to see the result


### Connectivity check
Check that the connectivity created from the tetrahedra coordinates is right with the one done by hand

In [12]:
def check(type):                                                                    
    cube_6_connectivity_sorted = sort_connectivity(cube_6_connectivity)
    print("cube_6_connectivity sorted:\n", cube_6_connectivity_sorted)
    tetrahedra_connectivity = add_connectivity(cube_6_tetrahedra)
    tetrahedra_connectivity_sorted = sort_connectivity(tetrahedra_connectivity)
    tetrahedra_connectivity_clean = clean(cube_connectivity, tetrahedra_connectivity_sorted)
    print("tetrahedra_connectivity_clean:\n", tetrahedra_connectivity_clean)

    # Correct!

# check("elia") # NBNBNBNB load the "elia" data!!!! Works only on mine


### Movie split
Now for the 6 tetrahedra split distanciate every tetrahedron so that you can understand better the triangles and the split. Do that based on a coefficient for the distance

In [13]:
def create_tetrahedra(atomList, name, file_name, frame):
    """create a frame of the movie. Here the tetrahedra are distant more and more based on the frame

    Args:
        atomList (List): list of tetrahedra vertices
        name (String): name of the general file 
        file_name (String): name of the specific file based on the frame
        frame (int): frame number

    Returns:
        Structure: structure of a molecule made of atoms disposed as a cube with tetrahedra that are distant more and more 
    """
    shifts_6_tetrahedra = [
        (-5.0, -5.0, 0.0),
        (-5.0, -5.0, -5.0),
        (5.0, 5.0, -5.0),
        (5.0, 5.0, 0.0),
        (5.0, 5.0, 5.0),
        (-5.0, -5.0, 5.0),
    ]

    shifts_5_tetrahedra = [
        (-10.0, -10.0, -10.0),
        (10.0, 10.0, -10.0),
        (10.0, -10.0, 10.0),
        (-10.0, 10.0, 10.0),
        (0.0, 0.0, 0.0),
    ]

    if name == "cube_5_tetrahedra":
        shift = shifts_5_tetrahedra
        n = 5
    elif name == "cube_6_tetrahedra":
        shift = shifts_6_tetrahedra
        n = 6
    else:
        print("Error: atomList must be either 'cube_5_tetrahedra' or 'cube_6_tetrahedra'")
        return
    
    structure = Structure.Structure(file_name)
    model = Model.Model(0)
    structure.add(model)
    for var in range(0, n):
        chain = Chain.Chain(str(var+1))
        model.add(chain)

        for i in atomList[var]:
            res = Residue.Residue((" ", i, " "), "PSE", " ")
            atom = Atom.Atom("PS" + str(i), tuple(map(lambda i,j: i+j, atoms["PS" + str(i)], tuple([float(frame/10) * i for i in shift[var]]))) , 1.0, 1.0, " ", "PS" + str(i), i, "C")
            res.add(atom)
            chain.add(res)
    
    return structure


def create_conn_per_tetrahedra():
    """ Same as before create the coupled fashion connection based on the vertices number

    Returns:
        List: list of couples of vertices
    """
    tetrahedra_connectivity = []
    for var in range(0,6):
        for i in range(4):
            for j in range(i+1, 4):
                tetrahedra_connectivity.append((var*4 + i+1, var *4 + j+1))
    return tetrahedra_connectivity

def create_movie(list_tetrahedra_vertices):
    """ Create movie

    Args:
        list_tetrahedra_vertices (List): list of thetrahedra vertices
    """
    if list_tetrahedra_vertices == "cube_5_tetrahedra":
        folder_name = "movie_5_conn/"
    elif list_tetrahedra_vertices == "cube_6_tetrahedra":
        folder_name = "movie_6_conn/"
    else:
        print("Error: list_tetrahedra_vertices must be either 'cube_5_tetrahedra' or 'cube_6_tetrahedra'")
        return
    
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    for frame in range(0, 11):
        file_name = str(frame)

        io = PDBIO()
        io.set_structure(create_tetrahedra(eval(list_tetrahedra_vertices), list_tetrahedra_vertices, file_name, frame))
        io.save(folder_name + file_name + ".pdb")

        # Read all lines
        with open(folder_name + file_name + ".pdb", "r") as file:
            lines = file.readlines()

        # Write back only lines that do NOT start with "TER" or "END"
        with open(folder_name + file_name + ".pdb", "w") as file:
            for line in lines:
                if not (line.startswith("TER") or line.startswith("END")):
                    file.write(line)

        #open stream
        with open(folder_name + file_name + ".pdb", "a") as stream:
            # Connect the cube vertices
            postprocess(stream, create_conn_per_tetrahedra())
            stream.write("END\n")


        print("PDB file with CONECT records saved as", folder_name + file_name + ".pdb")


In [14]:
name = "cube_5_tetrahedra" # Change the name between cube_5_tetrahedra and cube_6_tetrahedra
create_movie(name)

PDB file with CONECT records saved as movie_5_conn/0.pdb
PDB file with CONECT records saved as movie_5_conn/1.pdb
PDB file with CONECT records saved as movie_5_conn/2.pdb
PDB file with CONECT records saved as movie_5_conn/3.pdb
PDB file with CONECT records saved as movie_5_conn/4.pdb
PDB file with CONECT records saved as movie_5_conn/5.pdb
PDB file with CONECT records saved as movie_5_conn/6.pdb
PDB file with CONECT records saved as movie_5_conn/7.pdb
PDB file with CONECT records saved as movie_5_conn/8.pdb
PDB file with CONECT records saved as movie_5_conn/9.pdb
PDB file with CONECT records saved as movie_5_conn/10.pdb


In [15]:
print("Doing the rendering with disposition: ", type, " using ", name)  # Render the movie

for file_name in range(0, 4):


    if not (file_name == 0 or file_name == 5):
        continue
    
    folder_name = 0

    if name == "cube_5_tetrahedra":
        folder_name = "movie_5_conn/"
    elif name == "cube_6_tetrahedra":
        folder_name = "movie_6_conn/"
    else:
        print("Error: list_tetrahedra_vertices must be either 'cube_5_tetrahedra' or 'cube_6_tetrahedra'")

    with open((folder_name + str(file_name) + ".pdb"), "r") as f:
        pdb_data = f.read()

    # Visualize in Jupyter Notebook
    viewer = py3Dmol.view(width=600, height=600)
    viewer.addModel(pdb_data, "pdb")

    # # Style stick, add color
    # viewer.setStyle({"stick": {'colorscheme': "cyanCarbon"}})

    # Change the color based on the chain



    viewer.setStyle({'chain': 1}, {'stick': {'color': 'red'}})
    viewer.setStyle({'chain': 2}, {'stick': {'color': 'green'}})
    viewer.setStyle({'chain': 3}, {'stick': {'color': 'blue'}})
    viewer.setStyle({'chain': 4}, {'stick': {'color': 'yellow'}})
    viewer.setStyle({'chain': 5}, {'stick': {'color': 'purple'}})
    viewer.setStyle({'chain': 6}, {'stick': {'color': 'orange'}})


    viewer.zoomTo()
    # Change the angle
    viewer.rotate(45, "y", 0)
    viewer.rotate(65, "x", 0)
    viewer.rotate(45, "z", 0)
    viewer.show()




Doing the rendering with disposition:  jpn  using  cube_5_tetrahedra


3Dmol.js failed to load for some reason. Please check your browser console for error messages.